# Plan your trip with Kayak 📇

![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

**Scope of this project**🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

**Helpers**


Get weather data with an API:
*   https://nominatim.org/ or
*   https://openweathermap.org/appid

Save all the results in a .csv file with name of the cities, unique id.

Plot destinations on a map using plotly

Scrape Booking.com to obtain the informations:

*   hotel name
*   Url to booking.com page
*   Coordinates of latitude and longitude
*   Score given by the website users
*   Text description of the hotel

Create data lake using S3

ETL - create a SQL database using AWS RDS to extract data from S3 and store in new DB

**Deliverables**

1- A .csv file in an S3 bucket containing enriched information about weather and hotels for each French city

2- A SQL Database to get the cleaned data from S3

3- Two maps with TOP 5 destinations and TOP 20 hotels in the area

# Setup 💾 

In [2]:
#Importing libraries
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go

import requests
import boto3


# Scrapping 🔎

## 1 - Scrapping city names and coordinates - openstreetmap

In [63]:
%%capture
!python ../src/citylist.py

In [64]:
#let's load the city list from the json file
city_list=pd.read_csv('..\\data\\citylist.csv')
city_list.shape

(35, 1)

In [65]:
city_list.head()

,city
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen


In [66]:
# api to collect city informations and lats + longs
api_openstreet =[]
for city in city_list['city']:
  url='https://nominatim.openstreetmap.org/search?format=jsonv2&q=' + city + '&country=France'
  r1 = requests.get(url)
  api_openstreet.append(r1.json())

In [67]:
#gathering lat and long for each city in the dataset
latlon_list = []
for city,each in zip(city_list['city'],api_openstreet):
    latlon_list.append([city, float(each[0] ['lat']), float(each[0] ['lon'])])

#converting it to a pd dataframe
df0 = pd.DataFrame(latlon_list, columns = ['city','lat','lon'])
df0.head(5)


,city,lat,lon
0,Mont Saint Michel,48.635954,-1.511460
1,St Malo,48.649518,-2.026041
2,Bayeux,49.276462,-0.702474
3,Le Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966


In [68]:
#checkpoint
df0.to_csv("..\\data\\checkpoint1.csv", index=False)

## 2 - Scrapping weather data - openweathermap

Goal: Determine the cities where the weather will be the nicest within the next 7 days. For this, the following informations are collected:


* daily.pop - daily probability of rain
* daily.wind_speed - daily speed of wind
* daily.temp.day - daily temperature

Ranking of how nice weather will be based on a pointing system for these three critera:

1. distribute 10 points to each city in a normalized way by using average values in pop, wind_speed, temp.day
2. calculate the weighted average of points in three areas
    0.30 * ranking_pop + 0.30 * ranking_wind + 0.40 * temp_day

 Daily temperature should have a higher weight to decide whether it will be nice or not 😎

In [69]:
wk = open("..\\..\\.openweather","r").read()  #add your key here

In [70]:
#loading openweather api
api_openweather = []
for lat, lon in zip(df0['lat'], df0['lon']):
    base_url = 'https://api.openweathermap.org/data/2.5/onecall?'
    url = base_url + 'lat='+ str(lat) +'&lon='+ str(lon)+'&exclude=minutely,hourly'+\
                  '&units=metric' +'&appid='+wk
    api_openweather.append(requests.get(url).json())


Here are some weather information scrapped that will be considered to rank the cities on how nice will be the weather
* daily.pop : probability of precipitation. The values of the parameter vary between 0 and 1, where 0 is equal to 0%, 1 is equal to 100%

* daily.wind_speed Wind speed. Units – default: metre/sec, metric: metre/sec, imperial: miles/hour

* daily.temp Units – default: kelvin, metric: Celsius, imperial: Fahrenheit. How to change units used

units={units} 

    For temperature in Fahrenheit and wind speed in miles/hour, use units=imperial
    For temperature in Celsius and wind speed in meter/sec, use units=metric
    Temperature in Kelvin and wind speed in meter/sec is used by default


In [71]:
#here are the keys of daily keys
api_openweather[0]['daily'][0].keys()

dict_keys(['dt', 'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point', 'wind_speed', 'wind_deg', 'wind_gust', 'weather', 'clouds', 'pop', 'uvi'])

In [72]:
#naming column names
#following weather data is considered to rank how nice the weather will be
cols_wind =['d0_wind_speed', 'd1_wind_speed','d2_wind_speed',\
            'd3_wind_speed','d4_wind_speed','d5_wind_speed',\
            'd6_wind_speed','d7_wind_speed', 'avg_wind_speed']

cols_pop = ['d0_pop', 'd1_pop','d2_pop',\
            'd3_pop','d4_pop','d5_pop',\
            'd6_pop','d7_pop' ,'avg_pop']
            
cols_temp =['d0_temp', 'd1_temp','d2_temp',\
            'd3_temp','d4_temp','d5_temp',\
            'd6_temp','d7_temp' ,'avg_temp_day']

In [73]:
#api_openweather contains information for cities.
#it is not in a good format structure to use with pandas df
#it is neccessary to change it to a dataframe format suitable for analysis

#creating lists to collect data for all cities
wind_all = []
pop_all = []
temp_all = []

for cc in range (len(city_list)):
    #creating lists to collect neccessary infos for a single city
    wind_ = [api_openweather[cc]['daily'][ii]['wind_speed'] for ii in range(8)]
    pop_ = [api_openweather[cc]['daily'][ii]['pop'] for ii in range(8)]
    temp_ = [api_openweather[cc]['daily'][ii]['temp']['day'] for ii in range(8)]

    #appending mean of wind, pop, and temp
    wind_.append(round(np.mean(wind_),2))
    pop_.append(round(np.mean(pop_),3))
    temp_.append(round(np.mean(temp_),2))

    #appending them to global list
    wind_all.append(wind_)
    pop_all.append(pop_)
    temp_all.append(temp_)


In [74]:
#adding lists to dataframe
df0 [cols_wind] = wind_all
df0 [cols_pop] = pop_all
df0 [cols_temp] = temp_all

#sanity check
df0.head(3)

,city,lat,lon,d0_wind_speed,d1_wind_speed,d2_wind_speed,d3_wind_speed,d4_wind_speed,d5_wind_speed,d6_wind_speed,...,avg_pop,d0_temp,d1_temp,d2_temp,d3_temp,d4_temp,d5_temp,d6_temp,d7_temp,avg_temp_day
0,Mont Saint Michel,48.635954,-1.511460,7.21,14.15,13.22,7.62,6.59,6.02,11.53,...,0.426,18.17,18.42,15.86,18.10,20.32,20.69,16.73,17.00,18.16
1,St Malo,48.649518,-2.026041,7.78,14.96,15.10,8.02,8.56,7.03,12.82,...,0.350,18.15,17.97,16.77,17.69,19.94,19.63,16.52,17.03,17.96
2,Bayeux,49.276462,-0.702474,7.95,12.79,13.99,6.46,7.06,6.45,12.32,...,0.386,17.61,18.63,15.27,17.63,20.67,21.13,20.04,16.79,18.47


In [75]:
rank_cols = ['avg_wind_speed' , 'avg_pop', 'avg_temp_day']

#first, let's normalize the column values from 0 to 10
df_rank = df0[rank_cols]
df_rank = (df_rank-df_rank.min())/(df_rank.max()-df_rank.min()) * 10.
#for wind and pop columns, the lesser the value is the better the ranking should be. 
#fixing the values below.
df_rank[['avg_wind_speed' , 'avg_pop']] =  10. - df_rank[['avg_wind_speed' , 'avg_pop']]

#second, rank cities based on weighted sum below
df_rank['score'] = 0.30*df_rank['avg_wind_speed'] + 0.30*df_rank['avg_pop'] + 0.40*df_rank['avg_temp_day']

#third, add the rounded scores to global df
df0['score'] = round(df_rank['score'],2)

#let's overview results 
df0[['city','score']].head(5)


,city,score
0,Mont Saint Michel,1.34
1,St Malo,1.25
2,Bayeux,1.68
3,Le Havre,1.06
4,Rouen,3.61


*Sanity check* - Weather in St Malo and Mont St Micheal do not seem very ideal for tourism next week. Scores given above seem to work.

In [76]:
print('5 cities with the nicest weather next week:\n')

top5 = df0.sort_values(by='score',ascending=False).head(5)
top5.to_csv('..\\data\\top5city.csv',index=False)
top5

5 cities with the nicest weather next week:



,city,lat,lon,d0_wind_speed,d1_wind_speed,d2_wind_speed,d3_wind_speed,d4_wind_speed,d5_wind_speed,d6_wind_speed,...,d0_temp,d1_temp,d2_temp,d3_temp,d4_temp,d5_temp,d6_temp,d7_temp,avg_temp_day,score
31,Montauban,44.017584,1.354999,5.71,8.15,3.32,5.93,5.36,6.11,7.66,...,24.53,26.28,22.58,24.54,25.83,27.04,26.61,24.49,25.24,8.13
27,Collioure,42.525050,3.083155,4.82,5.29,5.23,2.07,3.81,4.55,6.64,...,22.91,24.36,26.54,22.93,21.13,23.88,24.36,21.61,23.46,8.09
33,Bayonne,43.494514,-1.473666,5.37,9.25,3.90,3.76,3.26,4.80,6.04,...,23.80,23.96,22.81,23.87,24.71,26.68,23.03,23.94,24.10,7.90
21,Aix en Provence,43.529842,5.447474,4.30,6.12,5.99,2.64,3.20,2.62,3.35,...,21.99,24.64,24.50,23.24,25.13,25.86,25.73,23.17,24.28,7.77
30,Toulouse,43.604462,1.444247,6.00,8.51,3.34,6.46,7.37,8.28,9.68,...,23.99,26.61,22.42,23.32,25.24,25.65,25.93,23.91,24.63,7.56


In [77]:
#adding a unique identifier
df0['city_id'] = [1000 + ii for ii in range(len(df0))]

In [78]:
#plotting a map with all city rankings
#all 35 destinations are plotted below to have an insight
fig1 = px.scatter_mapbox(
    df0,
    lat="lat",
    lon="lon",
    zoom = 4.6,
    color='avg_temp_day',
    size = 'score',
    title = 'Top 35 destinations with weather information',
    mapbox_style="open-street-map",#"outdoors",
    color_continuous_scale='jet',
    opacity = 0.9,
    height = 700,
    hover_name = 'city',
    hover_data=['score','avg_temp_day','avg_pop', 'avg_wind_speed']
    )

fig1.update_layout(hovermode="closest")

In [79]:
#saving a map with top 5 rankings
#all 35 destinations are plotted below to have an insight
fig1 = px.scatter_mapbox(
    df0.sort_values(by='score',ascending=False).head(5), #selecting top 5 cities 
    lat="lat",
    lon="lon",
    zoom = 4.6,
    color='avg_temp_day',
    size = 'score',
    title = 'Top 5 destinations with weather information',
    mapbox_style="open-street-map",#"outdoors",
    color_continuous_scale='jet',
    opacity = 0.9,
    height = 700,
    hover_name = 'city',
    hover_data=['score','avg_temp_day','avg_pop', 'avg_wind_speed']
    )

fig1.update_layout(hovermode="closest")
fig1.write_html("..\\deliverables\\weather_score_top_5_fr_cities.html")


In [80]:
#checkpoint 2 
df0.to_csv("..\\data\\checkpoint2.csv", index=False)

## 3 - Scrapping data from booking.com

Booking.com doesn't have aggregated databases so it is recommended to scrape data directly. 

The scrapped data should include the followings:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel

In [81]:
%%capture
!python ../src/bookinghotel.py

In [82]:
df_hotels=pd.read_csv('..\\data\\hotellist.csv')
df_hotels.shape

(125, 5)

In [83]:
df_hotels.head()

,hotel_city,hotel_name,hotel_score,hotel_desc,hotel_url
0,Bayonne,Villa la Renaissance,9.3,Deluxe Double Room,https://www.booking.com/hotel/fr/villa-la-rena...
1,Bayonne,Hôtel Villa KOEGUI Bayonne,9.0,Superior Room,https://www.booking.com/hotel/fr/villa-koegui-...
2,Bayonne,Hôtel Le Bayonne,7.8,Classic Double Room,https://www.booking.com/hotel/fr/le-bayonne-ha...
3,Bayonne,Temporesidence Cathedrale,7.9,Superior Studio,https://www.booking.com/hotel/fr/temporesidenc...
4,Bayonne,Okko Hotels Bayonne Centre,8.2,Classic Room,https://www.booking.com/hotel/fr/okko-hotels-b...


In [84]:
df_hotels["hotel_city"].value_counts()

Bayonne            25
Montauban          25
Aix-en-Provence    25
Toulouse           25
Collioure          25
Name: hotel_city, dtype: int64

Hotel list seems correct. It misses the lat and long of hotels. After reviewing Booking.com, I realized that they deleted the coordinates of listings from search result. They are available in each hotel's booking.com page. There is another round of scraping below.

In [88]:
%%capture
!python ../src/bookinghotel_latlons.py

In [89]:
df_hotels_latlon=pd.read_csv('..\\data\\hotellist_latlons.csv')
df_hotels_latlon.shape

(125, 3)

In [90]:
df_hotels_latlon.head()

,hotel_name,lat,lon
0,Hotel Loreak,43.493491,-1.452633
1,Hôtel Villa KOEGUI Bayonne,43.492371,-1.472588
2,Okko Hotels Bayonne Centre,43.495344,-1.482512
3,Temporesidence Cathedrale,43.489542,-1.478124
4,Hôtel Le Bayonne,43.485026,-1.475371


In [91]:
#now I have two different files for the hotel list. let's combine them
left = df_hotels
right = df_hotels_latlon
df1 =pd.merge(left,right,on=['hotel_name'])

In [92]:
#checkpoint 3
df1.to_csv("..\\data\\checkpoint3.csv", index=False)

In [93]:
top5city = pd.read_csv("..\\data\\top5city.csv")
top5city.head()
top5city =top5city.rename(columns={'score':'weather_score' , 'lat' : 'lat_city', 'lon':'lon_city'})

In [96]:
#let's merge weather scores in df1
df1['city'] = df1.hotel_city.str.replace('-',' ')
left = df1
right = top5city
df1 = pd.merge(left,right,on = 'city')


In [98]:
#I select top 20 hotels in top 5 cities below as requested by the project
df2 = pd.DataFrame ([], columns=df1.columns)
for city in df1['city'].unique():
    mask_ = df1['city'] == city
    df_add =  df1[mask_].sort_values(by='hotel_score',ascending=False).head(20)
    df2= pd.concat( [df2,df_add])

In [99]:
#re arranging df with respect to scores
df2 = df2.sort_values(by = ['weather_score', 'hotel_score'], ascending=False)

In [100]:
#outputing df to csv
df2.to_csv("..\\deliverables\\top20hotelstop5cities.csv", index=False)

In [101]:
#reading the last output
#df2=pd.read_csv("..\\deliverables\\top20hotelstop5cities.csv")

In [102]:
df2.hotel_score = df2.hotel_score.astype(float)
df2.weather_score = df2.weather_score.astype(float)
df2.columns

Index(['hotel_city', 'hotel_name', 'hotel_score', 'hotel_desc', 'hotel_url',
       'lat', 'lon', 'city', 'lat_city', 'lon_city', 'd0_wind_speed',
       'd1_wind_speed', 'd2_wind_speed', 'd3_wind_speed', 'd4_wind_speed',
       'd5_wind_speed', 'd6_wind_speed', 'd7_wind_speed', 'avg_wind_speed',
       'd0_pop', 'd1_pop', 'd2_pop', 'd3_pop', 'd4_pop', 'd5_pop', 'd6_pop',
       'd7_pop', 'avg_pop', 'd0_temp', 'd1_temp', 'd2_temp', 'd3_temp',
       'd4_temp', 'd5_temp', 'd6_temp', 'd7_temp', 'avg_temp_day',
       'weather_score'],
      dtype='object')

In [106]:
#plotting a map with top 20 hotels in top 5 cities
fig2 = px.scatter_mapbox(
    df2,
    lat="lat",
    lon="lon",
    zoom = 4.6,
    color = 'weather_score',
    size = 'hotel_score',
    title = 'Top 5 destinations with hotel and weather information',
    mapbox_style="open-street-map",#"outdoors",
    color_continuous_scale='jet',
    opacity = 0.9,
    height = 700,
    hover_name = 'city',
    hover_data=['hotel_name','hotel_score','hotel_desc','weather_score','avg_temp_day','avg_pop', 'avg_wind_speed']
    )

fig2.update_layout(hovermode="closest")
fig2.write_html("..\\deliverables\\weather_score_top_5_fr_cities_top_20_hotels.html")
fig2.show()

In [104]:
#checkpoint 4
df2.to_csv("..\\data\\checkpoint4.csv", index=False)

# S3 Bucket 📬

In [10]:
# creating AWS session using keys
kp0,kp1 = open("../../.karpuz","r").read().split('\n')  #add your key here
session = boto3.Session(
            aws_access_key_id=kp0, 
            aws_secret_access_key= kp1)
kp = []

s3_resource = session.resource('s3')
bucket_name = "awsbucketkayak"
kayak_bucket = s3_resource.create_bucket(Bucket=bucket_name)
                    #ACL = 'public-read')
# uploading
session = boto3.Session(aws_access_key_id=kp0, aws_secret_access_key=kp1)
kp0,kp1 = 0,0


In [15]:
s3 = session.client("s3")
#bucket = s3.create_bucket(Bucket=bucket_name)
s3.upload_file(Filename="..\\deliverables\\top20hotelstop5cities.csv",
                Bucket=bucket_name,
                Key='top20hotelstop5cities.csv')

OK. Uploaded!

In [7]:
# testing if data can be downloaded
fname= "https://awsbucketkayak.s3.amazonaws.com/top20hotelstop5cities.csv"
df_aws_booking = pd.read_csv(fname)
df_aws_booking.shape

(100, 38)

In [8]:
df_aws_booking.head()

,hotel_city,hotel_name,hotel_score,hotel_desc,hotel_url,lat,lon,city,lat_city,lon_city,...,d0_temp,d1_temp,d2_temp,d3_temp,d4_temp,d5_temp,d6_temp,d7_temp,avg_temp_day,weather_score
0,Montauban,L'Escale des Libellules by BNB Conciergerie Mtban,10.0,Apartment,https://www.booking.com/hotel/fr/escale-des-li...,44.016925,1.355524,Montauban,44.017584,1.354999,...,24.53,26.28,22.58,24.54,25.83,27.04,26.61,24.49,25.24,8.13
1,Montauban,Manoir de la Gravette Maison d'Hôtes,9.6,Deluxe Double Room,https://www.booking.com/hotel/fr/le-manoir-de-...,44.049043,1.343884,Montauban,44.017584,1.354999,...,24.53,26.28,22.58,24.54,25.83,27.04,26.61,24.49,25.24,8.13
2,Montauban,room five MONTAUBAN,9.5,One-Bedroom Apartment,https://www.booking.com/hotel/fr/room-five-mon...,44.017865,1.353859,Montauban,44.017584,1.354999,...,24.53,26.28,22.58,24.54,25.83,27.04,26.61,24.49,25.24,8.13
3,Montauban,Castel Bois Marie,9.3,Deluxe King Suite,https://www.booking.com/hotel/fr/castel-bois-m...,43.979174,1.309142,Montauban,44.017584,1.354999,...,24.53,26.28,22.58,24.54,25.83,27.04,26.61,24.49,25.24,8.13
4,Montauban,Les Chalets du Manoir,9.1,Double Room,https://www.booking.com/hotel/fr/les-chalets-d...,44.046593,1.306655,Montauban,44.017584,1.354999,...,24.53,26.28,22.58,24.54,25.83,27.04,26.61,24.49,25.24,8.13


**OK. It works!**

# RDS Database

In [24]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import psycopg2

import pandas as pd

In [31]:
end_point,un, uk= open("../../.kiraz","r").read().split('\n')  #add your key here
engine = create_engine(
    f'postgresql+psycopg2://{un}:{uk}@{end_point}', 
    echo = True)

Session = sessionmaker(bind = engine)
session = Session()
session

In [32]:
# retreiving csv from s3
fname= "https://awsbucketkayak.s3.amazonaws.com/top20hotelstop5cities.csv"
df_aws_booking = pd.read_csv(fname)

# pushing df s3 to sql db
df_aws_booking.to_sql("top20hotelstop5cities", engine)

2022-10-23 21:05:28,828 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-10-23 21:05:28,853 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-23 21:05:29,087 INFO sqlalchemy.engine.Engine select current_schema()
2022-10-23 21:05:29,088 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-23 21:05:29,366 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-10-23 21:05:29,367 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-23 21:05:29,609 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-10-23 21:05:29,611 INFO sqlalchemy.engine.Engine [generated in 0.00130s] {'name': 'top20hotelstop5cities'}
2022-10-23 21:05:30,092 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-23 21:05:30,097 INFO sqlalchemy.engine.Engine 
CREATE TABLE top20hotelstop5cities (
	index BIGINT, 
	hotel_city TEXT, 
	hotel_name TEXT, 
	hotel_score FLOAT(53), 
	hot

100

**Sucess**

*Please note that access to RDS is made only for my local ID*

In [35]:
df_aws_booking.head(1)

,hotel_city,hotel_name,hotel_score,hotel_desc,hotel_url,lat,lon,city,lat_city,lon_city,...,d0_temp,d1_temp,d2_temp,d3_temp,d4_temp,d5_temp,d6_temp,d7_temp,avg_temp_day,weather_score
0,Montauban,L'Escale des Libellules by BNB Conciergerie Mtban,10.0,Apartment,https://www.booking.com/hotel/fr/escale-des-li...,44.016925,1.355524,Montauban,44.017584,1.354999,...,24.53,26.28,22.58,24.54,25.83,27.04,26.61,24.49,25.24,8.13


In [42]:
conn = engine.connect()

# sql query below
query_db = """
SELECT hotel_city,hotel_name,hotel_score,weather_score,avg_temp_day as avgtemp 

FROM top20hotelstop5cities
ORDER BY weather_score DESC, hotel_score DESC, avgtemp DESC
"""

# outputing sql to dataframe
df_rds = pd.read_sql(query_db, engine)


2022-10-23 21:16:18,352 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-10-23 21:16:18,353 INFO sqlalchemy.engine.Engine [cached since 648.7s ago] {'name': '\nSELECT hotel_city,hotel_name,hotel_score,weather_score,avg_temp_day as avgtemp \n\nFROM top20hotelstop5cities\nORDER BY weather_score DESC, hotel_score DESC, avgtemp DESC\n'}
2022-10-23 21:16:18,835 INFO sqlalchemy.engine.Engine 
SELECT hotel_city,hotel_name,hotel_score,weather_score,avg_temp_day as avgtemp 

FROM top20hotelstop5cities
ORDER BY weather_score DESC, hotel_score DESC, avgtemp DESC

2022-10-23 21:16:18,837 INFO sqlalchemy.engine.Engine [raw sql] {}


In [43]:
df_rds

,hotel_city,hotel_name,hotel_score,weather_score,avgtemp
0,Montauban,L'Escale des Libellules by BNB Conciergerie Mtban,10.0,8.13,25.24
1,Montauban,Manoir de la Gravette Maison d'Hôtes,9.6,8.13,25.24
2,Montauban,room five MONTAUBAN,9.5,8.13,25.24
3,Montauban,Castel Bois Marie,9.3,8.13,25.24
4,Montauban,Les Chalets du Manoir,9.1,8.13,25.24
...,...,...,...,...,...
95,Toulouse,Privilège Appart Hôtel Saint Exupéry,8.1,7.56,24.63
96,Toulouse,Best Western Hotel Innes Toulouse Centre,8.1,7.56,24.63
97,Toulouse,"Crowne Plaza Toulouse, an IHG Hotel",8.0,7.56,24.63
98,Toulouse,Mercure Toulouse Centre Saint-Georges,8.0,7.56,24.63


In [44]:
df_rds.shape

(100, 5)

**OK. It is working**

# Summary

Weather and hotel information are scraped from web for top visited French citties.
Then, touristic French cities are ranked by their weather condition in the coming week. This notebook, at the end, provides 
* two maps plotted with TOP 5 destinations and TOP 20 hotels in the area
* creation of a .csv file in an S3 bucket with scrapped information about weather and hotels for top visited French citties
* creation of a SQL Database to get the cleaned data from S3, and 

--- End of Notebook ---
# END